# ginga-ml2-uncertainty-public — Turning Points PoC (Colab)

JSON（イベント列）をアップロードして、
中間の **ρ(t), G(t), 復元ρ̂(t)** を **保存（NPZ）**し、
**必要最低限の画像**で確認するノートです。


In [ ]:
# 1) clone repo
import os
repo = 'ginga-ml2-uncertainty-public'
if not os.path.exists(repo):
    !git clone -q https://github.com/Mokafe/ginga-ml2-uncertainty-public.git
%cd ginga-ml2-uncertainty-public


In [ ]:
# 2) install
!pip -q install -r requirements.txt


In [ ]:
# 3) upload your JSON (optional)
from google.colab import files
uploaded = files.upload()
user_json = next(iter(uploaded.keys())) if len(uploaded) else None
print('Uploaded JSON:', user_json)


In [ ]:
# 4) run (limited-but-complete export)
from src.poc_turning_points import run

json_path = user_json if user_json else 'data/sample/sample_g_1_reconstructed_keep_last9.json'

export_cfg = {
  'save_npz': True,
  'npz_dtype': 'float16',
  'save_g_mode': 'amp',
  'save_images': True,
  'image_kind': ['slice','mip'],
  'axis': 0,
  'slice_index': None,
  'select_frames': 'topk+ends',
  'k': 3,
  'also_ends': True,
  'max_frames': 6,
  'npz_name': 'intermediate.npz',
}

result = run(
    json_path=json_path,
    kimura2_txt='src/kimura2.txt',
    out_dir='assets',
    T=9, N=48,
    export_cfg=export_cfg,
)
result


In [ ]:
# 5) minimal display: overview + selected frames
from PIL import Image
from IPython.display import display

display(Image.open(result['out_png']))
print('frames_saved:', result['frames_saved'])
for kind, paths in result['image_paths'].items():
    print('kind:', kind, 'n=', len(paths))
    for p in paths:
        display(Image.open(p))


In [ ]:
# 6) verify NPZ content quickly
import numpy as np
npz = np.load(result['out_npz'])
print('NPZ keys:', list(npz.keys()))
print('rho_truth:', npz['rho_truth'].shape, npz['rho_truth'].dtype)
print('rho_rec  :', npz['rho_rec'].shape, npz['rho_rec'].dtype)
if 'G_amp' in npz:
    print('G_amp   :', npz['G_amp'].shape, npz['G_amp'].dtype)
else:
    print('G_real  :', npz['G_real'].shape, npz['G_real'].dtype)
    print('G_imag  :', npz['G_imag'].shape, npz['G_imag'].dtype)


In [ ]:
# 7) download outputs (optional)
from google.colab import files
import shutil

files.download('assets/intermediate.npz')
shutil.make_archive('assets_frames', 'zip', 'assets/frames')
files.download('assets_frames.zip')


## GitHubに保存（Colab UI）
- `File > Save a copy in GitHub...` を選んで保存します。
- Repository: `Mokafe/ginga-ml2-uncertainty-public`
- File path: `notebooks/00_turning_points_colab.ipynb`
- Branch: `main`
